In [1]:

import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split






In [2]:

seq_length=2
x_data_dim=4
batch_size=100
min_max_normalization_flag=True

data_dir = '../dataset'
fname = os.path.join(data_dir, 'data-02-stock_daily.csv')
df = pd.read_csv(fname)
dataset=df.copy()
ori_Y=dataset.pop("Close")
ori_X=dataset.copy()



In [3]:

X_train, X_test, Y_train, Y_test = train_test_split(ori_X,ori_Y, test_size=0.2, shuffle=False)
X_train, X_val, Y_train, Y_val= train_test_split(X_train,Y_train, test_size=0.2, shuffle=False)


In [4]:

## 데이터의 min , max, mean, std 값 구하기.
dataset_stats = X_train.describe()
dataset_stats = dataset_stats.transpose()

## data normalization
def min_max_norm(x):
  return (x - dataset_stats['min']) / (dataset_stats['max'] - dataset_stats['min'])

def standard_norm(x):
  return (x - dataset_stats['mean']) / dataset_stats['std']

if min_max_normalization_flag==True:
    min_max_norm_train_data = min_max_norm(X_train)
    min_max_norm_val_data = min_max_norm(X_val)
    min_max_norm_test_data = min_max_norm(X_test)

    data_gen_train=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_train_data.values.tolist(), Y_train.values.tolist(),
                                                                        length=seq_length, sampling_rate=1,
                                                                        batch_size=batch_size)
    data_gen_val=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_val_data.values.tolist(), Y_val.values.tolist(),
                                                                       length=seq_length, sampling_rate=1,
                                                                       batch_size=batch_size)
    data_gen_test=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_test_data.values.tolist(), Y_test.values.tolist(),
                                                                       length=seq_length, sampling_rate=1,
                                                                       batch_size=batch_size)
else:
    data_gen_train = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_train.values.tolist(),Y_train.values.tolist(),
                                                                   length=seq_length, sampling_rate=1,
                                                                   batch_size=batch_size)
    data_gen_val = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_val.values.tolist(),Y_val.values.tolist(),
                                                                   length=seq_length, sampling_rate=1,
                                                                   batch_size=batch_size)
    data_gen_test = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_test.values.tolist(),Y_test.values.tolist(),
                                                                        length=seq_length, sampling_rate=1,
                                                                        batch_size=batch_size)


In [5]:

## 모델
input_Layer = tf.keras.layers.Input(shape=(seq_length,x_data_dim))
x= tf.keras.layers.SimpleRNN(20,dropout = 0.1, recurrent_dropout = 0.5, return_sequences=True,activation='tanh')(input_Layer)
x= tf.keras.layers.SimpleRNN(10, dropout = 0.1, recurrent_dropout = 0.5,activation='tanh')(x)
x=tf.keras.layers.Dense(100,activation='relu')(x)
x=tf.keras.layers.Dense(50,activation='relu')(x)
Out_Layer=tf.keras.layers.Dense(1,activation=None)(x)

model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 4)]            0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 2, 20)             500       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 10)                310       
_________________________________________________________________
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 7,011
Trainable params: 7,011
Non-trainable params: 0
____________________________________________________

In [6]:

loss_function=tf.keras.losses.mean_squared_error
optimize=tf.keras.optimizers.Adam(learning_rate=0.001)
metric=tf.keras.metrics.mean_absolute_error
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])

history = model.fit(
      data_gen_train,
      validation_data=data_gen_val,
      steps_per_epoch=len(X_train)/batch_size,
      epochs=500,
      validation_freq=1
)
print(model.evaluate(data_gen_test))


Epoch 1/500
5/4 [================================] - 3s 526ms/step - loss: 501046.7188 - mean_absolute_error: 702.7700 - val_loss: 288175.0000 - val_mean_absolute_error: 536.5043
Epoch 2/500
5/4 [================================] - 1s 128ms/step - loss: 500550.4062 - mean_absolute_error: 702.4252 - val_loss: 288046.4062 - val_mean_absolute_error: 536.3850
Epoch 3/500
5/4 [================================] - 1s 181ms/step - loss: 500072.0625 - mean_absolute_error: 702.0935 - val_loss: 287878.7812 - val_mean_absolute_error: 536.2295
Epoch 4/500
5/4 [================================] - 1s 111ms/step - loss: 499458.4062 - mean_absolute_error: 701.6719 - val_loss: 287647.4375 - val_mean_absolute_error: 536.0150
Epoch 5/500
5/4 [================================] - 1s 141ms/step - loss: 498759.6250 - mean_absolute_error: 701.1839 - val_loss: 287314.1250 - val_mean_absolute_error: 535.7061
Epoch 6/500
5/4 [================================] - 1s 107ms/step - loss: 497825.7500 - mean_absolute_er

5/4 [================================] - 2s 405ms/step - loss: 6677.1436 - mean_absolute_error: 68.1050 - val_loss: 23513.1660 - val_mean_absolute_error: 152.8426
Epoch 48/500
5/4 [================================] - 1s 185ms/step - loss: 6383.8311 - mean_absolute_error: 66.6484 - val_loss: 22949.5996 - val_mean_absolute_error: 150.9855
Epoch 49/500
5/4 [================================] - 1s 139ms/step - loss: 6382.9053 - mean_absolute_error: 66.8832 - val_loss: 22633.3652 - val_mean_absolute_error: 149.9142
Epoch 50/500
5/4 [================================] - 1s 250ms/step - loss: 6500.9331 - mean_absolute_error: 67.6221 - val_loss: 21000.8887 - val_mean_absolute_error: 144.2950
Epoch 51/500
5/4 [================================] - 1s 154ms/step - loss: 6150.8809 - mean_absolute_error: 66.5844 - val_loss: 20088.8320 - val_mean_absolute_error: 140.9932
Epoch 52/500
5/4 [================================] - 1s 120ms/step - loss: 6074.9141 - mean_absolute_error: 66.7048 - val_loss: 1910

5/4 [================================] - 1s 176ms/step - loss: 2309.0713 - mean_absolute_error: 38.4100 - val_loss: 4650.5239 - val_mean_absolute_error: 55.7589
Epoch 95/500
5/4 [================================] - 1s 148ms/step - loss: 2529.2029 - mean_absolute_error: 40.2025 - val_loss: 4475.4072 - val_mean_absolute_error: 54.6535
Epoch 96/500
5/4 [================================] - 1s 101ms/step - loss: 2334.0498 - mean_absolute_error: 39.0440 - val_loss: 4364.1836 - val_mean_absolute_error: 54.0928
Epoch 97/500
5/4 [================================] - 1s 179ms/step - loss: 2537.7087 - mean_absolute_error: 39.9540 - val_loss: 4346.6162 - val_mean_absolute_error: 54.8205
Epoch 98/500
5/4 [================================] - 1s 116ms/step - loss: 2487.0303 - mean_absolute_error: 40.2629 - val_loss: 4286.6621 - val_mean_absolute_error: 55.7466
Epoch 99/500
5/4 [================================] - 1s 133ms/step - loss: 2417.2322 - mean_absolute_error: 39.7469 - val_loss: 4091.5327 - va

Epoch 141/500
5/4 [================================] - 1s 130ms/step - loss: 1843.8107 - mean_absolute_error: 34.4538 - val_loss: 1447.9683 - val_mean_absolute_error: 33.9627
Epoch 142/500
5/4 [================================] - 1s 214ms/step - loss: 1739.0201 - mean_absolute_error: 34.1003 - val_loss: 1383.5100 - val_mean_absolute_error: 33.1773
Epoch 143/500
5/4 [================================] - 1s 242ms/step - loss: 1861.1140 - mean_absolute_error: 34.9289 - val_loss: 1342.0924 - val_mean_absolute_error: 32.7162
Epoch 144/500
5/4 [================================] - 1s 135ms/step - loss: 1728.6735 - mean_absolute_error: 33.3373 - val_loss: 1337.2363 - val_mean_absolute_error: 32.7425
Epoch 145/500
5/4 [================================] - 1s 126ms/step - loss: 1761.2909 - mean_absolute_error: 33.1910 - val_loss: 1307.2188 - val_mean_absolute_error: 32.3794
Epoch 146/500
5/4 [================================] - 1s 111ms/step - loss: 1764.1492 - mean_absolute_error: 33.4530 - val_l

5/4 [================================] - 0s 56ms/step - loss: 1650.4023 - mean_absolute_error: 31.5152 - val_loss: 631.8827 - val_mean_absolute_error: 22.5946
Epoch 189/500
5/4 [================================] - 0s 58ms/step - loss: 1518.5476 - mean_absolute_error: 31.2773 - val_loss: 654.7717 - val_mean_absolute_error: 22.9868
Epoch 190/500
5/4 [================================] - 0s 53ms/step - loss: 1462.3420 - mean_absolute_error: 29.8464 - val_loss: 657.0067 - val_mean_absolute_error: 23.0047
Epoch 191/500
5/4 [================================] - 0s 54ms/step - loss: 1347.7761 - mean_absolute_error: 29.0040 - val_loss: 666.0685 - val_mean_absolute_error: 23.1441
Epoch 192/500
5/4 [================================] - 0s 56ms/step - loss: 1440.6272 - mean_absolute_error: 30.4964 - val_loss: 608.1348 - val_mean_absolute_error: 22.1025
Epoch 193/500
5/4 [================================] - 0s 52ms/step - loss: 1366.5254 - mean_absolute_error: 29.0335 - val_loss: 649.2230 - val_mean_

5/4 [================================] - 0s 73ms/step - loss: 1015.4894 - mean_absolute_error: 25.4081 - val_loss: 68.2186 - val_mean_absolute_error: 7.2892
Epoch 416/500
5/4 [================================] - 0s 73ms/step - loss: 925.4828 - mean_absolute_error: 23.0584 - val_loss: 44.2253 - val_mean_absolute_error: 5.7021
Epoch 417/500
5/4 [================================] - 0s 70ms/step - loss: 1099.9230 - mean_absolute_error: 24.9525 - val_loss: 75.4896 - val_mean_absolute_error: 7.7180
Epoch 418/500
5/4 [================================] - 0s 64ms/step - loss: 921.6935 - mean_absolute_error: 23.6701 - val_loss: 82.0140 - val_mean_absolute_error: 8.0846
Epoch 419/500
5/4 [================================] - 0s 58ms/step - loss: 953.3729 - mean_absolute_error: 23.6543 - val_loss: 66.9684 - val_mean_absolute_error: 7.2069
Epoch 420/500
5/4 [================================] - 0s 62ms/step - loss: 968.7016 - mean_absolute_error: 24.6056 - val_loss: 77.8456 - val_mean_absolute_error:

5/4 [================================] - 0s 63ms/step - loss: 1039.0946 - mean_absolute_error: 24.5754 - val_loss: 88.6375 - val_mean_absolute_error: 8.3406
Epoch 464/500
5/4 [================================] - 0s 60ms/step - loss: 958.0270 - mean_absolute_error: 24.3762 - val_loss: 59.4149 - val_mean_absolute_error: 6.6491
Epoch 465/500
5/4 [================================] - 0s 59ms/step - loss: 863.9806 - mean_absolute_error: 22.9935 - val_loss: 54.7972 - val_mean_absolute_error: 6.3473
Epoch 466/500
5/4 [================================] - 0s 55ms/step - loss: 950.0157 - mean_absolute_error: 24.2397 - val_loss: 79.6009 - val_mean_absolute_error: 7.8656
Epoch 467/500
5/4 [================================] - 0s 63ms/step - loss: 956.4760 - mean_absolute_error: 23.9445 - val_loss: 65.5640 - val_mean_absolute_error: 7.0452
Epoch 468/500
5/4 [================================] - 0s 68ms/step - loss: 957.2887 - mean_absolute_error: 24.3483 - val_loss: 41.4336 - val_mean_absolute_error: 

In [ ]:
test_data_X, test_data_Y=data_gen_test[0]
prediction_Y=model.predict(test_data_X).flatten()
Y_test=test_data_Y.flatten()

visual_y=[]
visual_pre_y=[]
for i in range(len(prediction_Y)):
    label = Y_test[i]
    prediction = prediction_Y[i]
    print("실제가격: {:.3f}, 예상가격: {:.3f}".format(label, prediction))
    visual_y.append(label)
    visual_pre_y.append(prediction)

time = range(1, len(visual_y) + 1)
plt.plot(time, visual_y, 'r', label='ture')
plt.plot(time, visual_pre_y, 'b', label='prediction')
plt.title('stock prediction')
plt.xlabel('time')
plt.ylabel('value')
plt.legend()
plt.show()